In [ ]:
# Update LangChain to latest version
!pip install -U langchain==0.1.20

In [5]:
# Refer to https://python.langchain.com/v0.1/docs/integrations/toolkits/github/
import os

from langchain.agents import AgentType, initialize_agent
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_community.chat_models import ChatOpenAI

In [15]:
# Load env vars from .env file
from dotenv import load_dotenv
load_dotenv()

os.environ["GITHUB_APP_ID"] = os.getenv("GITHUB_APP_ID")
os.environ["GITHUB_APP_PRIVATE_KEY"] = os.getenv("GITHUB_APP_PRIVATE_KEY")
os.environ["GITHUB_REPOSITORY"] = os.getenv("GITHUB_REPOSITORY")
os.environ["GITHUB_BRANCH"] = os.getenv("GITHUB_BRANCH")
os.environ["GITHUB_BASE_BRANCH"] = os.getenv("GITHUB_BASE_BRANCH")

# This example also requires an OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

In [20]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)
tools = toolkit.get_tools()

# STRUCTURED_CHAT includes args_schema for each tool, helps tool args parsing errors.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print("Available tools:")
for tool in tools:
    print("\t" + tool.name)

Available tools:
	Get Issues
	Get Issue
	Comment on Issue
	List open pull requests (PRs)
	Get Pull Request
	Overview of files included in PR
	Create Pull Request
	List Pull Requests' Files
	Create File
	Read File
	Update File
	Delete File
	Overview of existing files in Main branch
	Overview of files in current working branch
	List branches in this repository
	Set active branch
	Create a new branch
	Get files from a directory
	Search issues and pull requests
	Search code
	Create review request


In [ ]:
agent.run(
    "You have the software engineering capabilities of a Google Principle engineer. You are tasked with completing issues on a github repository. Please look at the existing issues and complete them."
)
# error TypeError: GitHubAction._run() got an unexpected keyword argument 'issue_number', update the prompt to "describe" instead of "complete" issues will bypass the error or use a more detailed prompt to skip the issue found process in the repository.

In [26]:
from github import Github

# Authentication is defined via github.Auth
from github import Auth

# using an access token
auth = Auth.Token(os.getenv("GITHUB_PAT"))

# Public Web Github
g = Github(auth=auth)


In [27]:
def fix_issues(repo_owner, repo_name):
    repo = g.get_repo(f"{repo_owner}/{repo_name}")
    
    # Get existing issues
    issues = repo.get_issues(state="open")
    for issue in issues:
        print(f"Issue #{issue.number}: {issue.title}")
        
        # Use OpenAI to generate a fix for the issue
        prompt = f"Fix the following issue in the {repo_name} repository:\n{issue.title}\n{issue.body}"
        response = agent.run(prompt)
        
        # Create a new branch for the fix
        branch_name = f"fix-issue-{issue.number}"
        repo.create_git_ref(f"refs/heads/{branch_name}", repo.get_branch("main").commit.sha)
        
        # Commit the changes to the new branch
        # (Assuming the changes are in a file called 'fix.py')
        fix_content = response
        repo.create_file("fix.py", f"Fix for issue #{issue.number}", fix_content, branch=branch_name)
        
        # Create a pull request for the fix
        pr_title = f"Fix for issue #{issue.number}"
        pr_body = f"This pull request fixes issue #{issue.number}:\n{issue.title}"
        repo.create_pull(title=pr_title, body=pr_body, head=branch_name, base="main")

In [28]:
# Note error will occur in the final step since fine-grained token only granted with Public Repositories (read-only) access for this example, but still the new branch and PR will be created and can be manually merged.
fix_issues("yike5460", "justNotes")

Issue #1: Update the README file 


> Entering new AgentExecutor chain...
Thought: The user has requested to update the README file by appending new content. The first step is to read the existing content of the README file.

Action:
```json
{
  "action": "Read File",
  "action_input": "README.md"
}
```
Observation: # Notes not yet organized

## Table of Contents
- [Introduction](#introduction)
- [Infrastructure](#infrastructure)
  - [UT Pipeline](#ut-pipeline)
  - [API Design](#api-design)
  - [Container](#container)
  - [GPU](#gpu)
- [GeneativeAI](#geneativeai)
  - [txt2txt](#txt2txt)
    - [Bert](#bert)
    - [Claude](#claude)
    - [LCEL](#lcel)
    - [RAG](#rag)
  - [txt2img2vid](#txt2img2vid)
    - [LoRA](#lora)
    - [CompyUI](#compyui)
    - [sd3dalle](#sd3dalle)
- [DevOps](#devops)
  - [Custom workflow](#custom-workflow)
  - [Repos sync](#repos-sync)
- [Contributing](#contributing)
- [License](#license)

## Introduction
"The best way to learn something is to build it." someone

Request POST /repos/yike5460/justNotes/git/refs failed with 403: Forbidden


GithubException: 403 {"message": "Resource not accessible by personal access token", "documentation_url": "https://docs.github.com/rest/git/refs#create-a-reference"}